In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn as nn
import helpers.configs as configs

from torch.utils.data import DataLoader, random_split
from helpers.preprocess import Task2Dataset

## 1. Dataset

In [17]:
BATCH_SIZE = 2
TEST_SPLIT_RATIO = 0.1
SEED = 42

In [8]:
# run data_clean.py first to convert .csv files to .pkl files
train_filename = "data/train.pkl"
test_filename = "data/test.pkl"
train_dataset = Task2Dataset(train_filename)
test_dataset = Task2Dataset(test_filename)
len(train_dataset), len(test_dataset)

(18995, 12664)

In [23]:
num_val_samples = int(len(train_dataset) * TEST_SPLIT_RATIO)
train_dataset, val_dataset = random_split(train_dataset, [len(train_dataset) - num_val_samples, num_val_samples],
                                         generator=torch.Generator().manual_seed(SEED))
len(train_dataset), len(val_dataset)

(15387, 1709)

In [11]:
# train_dataset[0]

In [12]:
# test_dataset[0]

In [24]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [89]:
for X_ages, X_features, y_cls, y_reg in train_loader:
    print(f"{X_ages.shape}, {X_features.shape}, {y_cls.shape}, {y_reg.shape}")
    print(f"{y_cls.dtype}")
    break

torch.Size([2]), torch.Size([2, 12, 34]), torch.Size([2, 1, 11]), torch.Size([2, 1, 4])
torch.float64


## 2. Model

In [62]:
from helpers.models import SequenceModel

In [63]:
seq_model = SequenceModel(configs.conf_seq_model_params).to(configs.conf_device)

In [64]:
for X_ages, X_features, y_cls, y_reg in train_loader:
    print(f"{X_ages.shape}, {X_features.shape}, {y_cls.shape}, {y_reg.shape}")
    break

torch.Size([2]), torch.Size([2, 12, 34]), torch.Size([2, 1, 11]), torch.Size([2, 1, 4])


In [68]:
X_ages, X_features = X_ages.float().to(configs.conf_device), X_features.float().to(configs.conf_device)
X_cls, X_reg = seq_model(X_ages, X_features, None)
X_cls.shape, X_reg.shape

(torch.Size([2, 11]), torch.Size([2, 4]))

In [58]:
# nn.Transformer.generate_square_subsequent_mask(2)

## 3. Loss Functions

In [59]:
from helpers.utils import r2_loss

In [61]:
y_pred = torch.randn(BATCH_SIZE)
y_true = torch.randn(BATCH_SIZE)
r2_loss_val = r2_loss(y_pred, y_true)
r2_loss_val

tensor(-0.5300)

In [74]:
from helpers.utils import auc_star_loss

In [88]:
y_pred = torch.sigmoid(torch.randn(BATCH_SIZE))
y_true = (torch.rand(BATCH_SIZE) > 0.5).long()
epoch_pred = torch.sigmoid(torch.randn(5 * BATCH_SIZE))
epoch_true = (torch.randn(5 * BATCH_SIZE) > 0.5).long()
auc_loss_val = auc_star_loss(y_pred, y_true, epoch_pred, epoch_true)
auc_loss_val

tensor(-0.2091)

## 4. Training

In [3]:
import time

time_stamp = f"{time.time()}".replace(".", "_")
params = {}
bash_params = {
    "epochs": 150,
    "batch_size": 1024,
    "test_split_ratio": 0.1,
    "seed": 42,
    "cls_weight": 20,
    "last_cls_weight": 15,
    "log_dir": f"run/{time_stamp}",
    "model_save_dir": f"params/{time_stamp}",
    "if_notebook": True
}
params.update(bash_params)
params.update(configs.conf_auc_params)
params.update(configs.conf_seq_model_opt_params)
params

{'epochs': 150,
 'batch_size': 1024,
 'test_split_ratio': 0.1,
 'seed': 42,
 'cls_weight': 20,
 'last_cls_weight': 15,
 'log_dir': 'run/1649433880_883521',
 'model_save_dir': 'params/1649433880_883521',
 'if_notebook': True,
 'auc_p': 2,
 'auc_gamma': 0.7,
 'constructor': torch.optim.adamw.AdamW,
 'opt_params': {'lr': 0.002}}

In [4]:
from torch.utils.data import DataLoader
from helpers.preprocess import Task2Dataset
from helpers.models import SequenceModel
from helpers.trainers import SeqModelTrainer

In [5]:
train_filename = "data/train.pkl"
test_filename = "data/test.pkl"
train_dataset = Task2Dataset(train_filename)
test_dataset = Task2Dataset(test_filename)
num_val_samples = int(len(train_dataset) * params["test_split_ratio"])
train_dataset, val_dataset = random_split(train_dataset, [len(train_dataset) - num_val_samples, num_val_samples],
                                         generator=torch.Generator().manual_seed(params["seed"]))
train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=params["batch_size"])
test_loader = DataLoader(test_dataset, batch_size=params["batch_size"])

seq_model = SequenceModel(configs.conf_seq_model_params).to(configs.conf_device)

In [6]:
configs.conf_seq_model_params

{'input_dim': 34,
 'transformer_encoder_layer_params': {'d_model': 64,
  'nhead': 8,
  'dim_feedforward': 64},
 'transformer_encoder_params': {'num_layers': 3}}

In [7]:
trainer = SeqModelTrainer(seq_model, train_loader, val_loader, params)

In [8]:
trainer.train()

epochs:   0%|          | 0/150 [00:00<?, ?it/s]

eval:   0%|          | 0/2 [00:00<?, ?it/s]

training:   0%|          | 0/17 [00:00<?, ?it/s]

eval:   0%|          | 0/2 [00:00<?, ?it/s]

training:   0%|          | 0/17 [00:00<?, ?it/s]

...

eval:   0%|          | 0/2 [00:00<?, ?it/s]

training:   0%|          | 0/17 [00:00<?, ?it/s]

eval:   0%|          | 0/2 [00:00<?, ?it/s]

training:   0%|          | 0/17 [00:00<?, ?it/s]

eval:   0%|          | 0/2 [00:00<?, ?it/s]